In [1]:
import mbuild as mb
import foyer
import hoomd
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

In [6]:
PCPDTFBT_C1_BO = mb.load("/Users/madilyn/Projects/repos/forcefields/monomers/PCPDTFBT_C1_BO_typed.mol2")

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mdtraj/core/trajectory.py:438: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')


In [7]:
esp_ff = foyer.Forcefield(forcefield_files="/Users/madilyn/Projects/repos/forcefields/PCPDTFBT_C1_BO.xml")

# We have to add the underscore to the names manually if we are using foyer XML files without SMARTS definitions
PCPDTFBT_C1_BO = mb.load("/Users/madilyn/Projects/repos/forcefields/monomers/PCPDTFBT_C1_BO_typed.mol2")

for p in PCPDTFBT_C1_BO.particles():
    p.name = f"_{p.name}"

box = mb.fill_box(compound=PCPDTFBT_C1_BO, n_compounds=1, box=[10,10,10])
box_pmd = esp_ff.apply(box)

snapshot, forcefield, refs = create_hoomd_forcefield(box_pmd, auto_scale=True, r_cut=2.5)




#setting cpu and simulation
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snapshot)

#setting the integrator
kt = 1.2
free_particle = hoomd.filter.Tags(tags=[0,1])  #letting hoomd know which particles to update the positions of
integrator = hoomd.md.Integrator(dt = 0.0001)
nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), kT=1.2, tau=0.5) #what is tau?
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=free_particle, kT=kt)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_CPDT.gsd",  #name the output file
    mode="wb"
)
sim.operations.writers.append(gsd_writer)



/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H79
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C19
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C49
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C44
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H64
  warnings.warn(
/Users/madilyn/miniconda3/envs/espa

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C1 pos=([1.5208 2.5323 7.3548]), 4 bonds, id: 6359537648> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C2 pos=([1.6172 2.4313 7.2863]), 3 bonds, id: 6359536448> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C2 pos=([1.6442 2.4844 7.1421]), 3 bonds, id: 6359536736> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_S1 pos=([1.692  2.3415 7.0394]), 2 bonds, id: 6359536640> does not ha

FoyerError: Found no types for atom 3 (None).